In [265]:
import requests
import pandas as pd
from datetime import datetime

In [266]:
api_key = "0d1c5d373e2b4cb0b07152200250909"
baseurl = "http://api.weatherapi.com/v1/"
# f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"
city="Louisville"

In [267]:
response = requests.get(f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}")
response.status_code

200

In [268]:
data = response.json()
data['current']

{'last_updated_epoch': 1757442600,
 'last_updated': '2025-09-09 14:30',
 'temp_c': 25.1,
 'temp_f': 77.2,
 'is_day': 1,
 'condition': {'text': 'Sunny',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
  'code': 1000},
 'wind_mph': 2.5,
 'wind_kph': 4.0,
 'wind_degree': 103,
 'wind_dir': 'ESE',
 'pressure_mb': 1022.0,
 'pressure_in': 30.17,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 37,
 'cloud': 0,
 'feelslike_c': 25.1,
 'feelslike_f': 77.1,
 'windchill_c': 27.4,
 'windchill_f': 81.3,
 'heatindex_c': 26.9,
 'heatindex_f': 80.5,
 'dewpoint_c': 10.8,
 'dewpoint_f': 51.4,
 'vis_km': 16.0,
 'vis_miles': 9.0,
 'uv': 6.7,
 'gust_mph': 2.8,
 'gust_kph': 4.6,
 'short_rad': 588.45,
 'diff_rad': 83.04,
 'dni': 1311.49,
 'gti': 327.28}

In [269]:
forecast_url = f"{baseurl}forecast.json?key={api_key}&q={city}&days=3"
forecast_response = requests.get(forecast_url)
forecast_response.status_code

200

In [270]:
forecast_data = forecast_response.json()
forecast = forecast_data['forecast']['forecastday']

In [271]:
forecast_3_days_summary = []
for day in forecast:
    forecast_3_days_summary.append(f"{day['date']}: High {day['day']['maxtemp_f']}F, Low {day['day']['mintemp_f']}F - {day['day']['condition']['text']}")

In [272]:
forecast_3_days_summary

['2025-09-09: High 83.8F, Low 53.4F - Sunny',
 '2025-09-10: High 83.3F, Low 59.5F - Cloudy ',
 '2025-09-11: High 86.2F, Low 64.8F - Sunny']

In [273]:
forecast_3_days_summary = []
for day in forecast:
    date_obj = datetime.strptime(day['date'], '%Y-%m-%d')
    formatted_date = date_obj.strftime('%A, %B %#d')
    forecast_3_days_summary.append(f"{formatted_date}:  High {day['day']['maxtemp_f']}F / Low {day['day']['mintemp_f']}F - {day['day']['condition']['text']} ")

forecast_3_days_summary

['Tuesday, September 9:  High 83.8F / Low 53.4F - Sunny ',
 'Wednesday, September 10:  High 83.3F / Low 59.5F - Cloudy  ',
 'Thursday, September 11:  High 86.2F / Low 64.8F - Sunny ']

In [274]:
today = pd.Timestamp.now().strftime("%Y-%m-%d")
astronomy_url = f"{baseurl}astronomy.json?key={api_key}&q={city}&dt={today}"


In [275]:
# --- Current Weather ---
def get_city_data(city, api_key):
    current_url = f"{baseurl}current.json?key={api_key}&q={city}"
    current_response = requests.get(current_url)
    current_data = current_response.json()["current"]

In [276]:
# --- 3-Day Forecast ---
def get_city_data(city, api_key):
    forecast_url = f"{baseurl}forecast.json?key={api_key}&q={city}&days=3"
    forecast_response = requests.get(forecast_url)
    forecast = forecast_response.json()["forecast"]["forecastday"]

    forecast_summary = []
    for day in forecast:
        date_obj = datetime.strptime(day["date"], "%Y-%m-%d")
        formatted_date = date_obj.strftime("%A, %B %#d") 
        forecast_3_days_summary.append(
            f"{formatted_date}: High {day['day']['maxtemp_f']}°F / "
            f"Low {day['day']['mintemp_f']}°F - {day['day']['condition']['text']}"
        )

In [277]:
# --- Astronomy Data ---
def get_city_data(city, api_key):
    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    astronomy_url = f"{baseurl}astronomy.json?key={api_key}&q={city}&dt={today}"
    astronomy_response = requests.get(astronomy_url)
    astronomy = astronomy_response.json()["astronomy"]["astro"]

In [278]:
# --- Build Summary Dictionary ---
def get_city_data(city, api_key):
    summary = {
        "city": city,
        "current": {
            "Temperature": data['current']['temp_f'],
            "Condition": data['current']['condition']['text'],
            "Humidity": data['current']['humidity'],
            "Wind Speed": data['current']['wind_mph']
        },
        "forecast": forecast_3_days_summary,
        "astronomy": {
            "sunrise": astronomy["sunrise"],
            "sunset": astronomy["sunset"],
            "moon_phase": astronomy["moon_phase"]
        }
    }

    return summary

In [280]:
def get_city_data(city, api_key):
    current_url = f"{baseurl}current.json?key={api_key}&q={city}&aqi=yes"
    current_response = requests.get(current_url).json()
    current_data = current_response["current"]

    forecast_url = f"{baseurl}forecast.json?key={api_key}&q={city}&days=3"
    forecast_response = requests.get(forecast_url)
    forecast = forecast_response.json()["forecast"]["forecastday"]

    forecast_summary = []
    for day in forecast:
        date_obj = datetime.strptime(day["date"], "%Y-%m-%d")
        formatted_date = date_obj.strftime("%A, %B %#d")  # use %#d on Windows, %-d on Mac/Linux
        forecast_summary.append(
            f"{formatted_date}: High {day['day']['maxtemp_f']}°F / "
            f"Low {day['day']['mintemp_f']}°F - {day['day']['condition']['text']}"
        )


    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    astronomy_url = f"{baseurl}astronomy.json?key={api_key}&q={city}&dt={today}"
    astronomy_response = requests.get(astronomy_url)
    astronomy = astronomy_response.json()["astronomy"]["astro"]

    air_quality = current_response["current"].get("air_quality", {})

    summary = {
        "city": city,
        "current": {
            "Temperature": data['current']['temp_f'],
            "Condition": data['current']['condition']['text'],
            "Humidity": data['current']['humidity'],
            "Wind Speed": data['current']['wind_mph']
        },
        "forecast": forecast_3_days_summary,
        "astronomy": {
            "sunrise": astronomy["sunrise"],
            "sunset": astronomy["sunset"],
            "moon_phase": astronomy["moon_phase"]
        },
        "air_quality": {
            "PM2.5": air_quality.get("pm2_5"),
            "PM10": air_quality.get("pm10"),
            "O3": air_quality.get("o3"),
            "NO2": air_quality.get("no2"),
            "SO2": air_quality.get("so2"),
            "CO": air_quality.get("co")
        }
    }

    return summary

city_weather = get_city_data("Louisville", api_key)

current_df = pd.DataFrame([city_weather["current"]])
current_df["City"] = city_weather["city"]
print("=== Current Weather ===")
print(current_df, "\n")

forecast_df = pd.DataFrame({"Forecast": city_weather["forecast"]})
forecast_df["City"] = city_weather["city"]
print("=== 3-Day Forecast ===")
print(forecast_df, "\n")

astronomy_df = pd.DataFrame([city_weather["astronomy"]])
astronomy_df["City"] = city_weather["city"]
print("=== Astronomy ===")
print(astronomy_df, "\n")

air_quality_df = pd.DataFrame([city_weather["air_quality"]])
air_quality_df["City"] = city_weather["city"]
print("=== Air Quality ===")
print(air_quality_df, "\n")


=== Current Weather ===
   Temperature Condition  Humidity  Wind Speed        City
0         77.2     Sunny        37         2.5  Louisville 

=== 3-Day Forecast ===
                                            Forecast        City
0  Tuesday, September 9:  High 83.8F / Low 53.4F ...  Louisville
1  Wednesday, September 10:  High 83.3F / Low 59....  Louisville
2  Thursday, September 11:  High 86.2F / Low 64.8...  Louisville 

=== Astronomy ===
    sunrise    sunset      moon_phase        City
0  07:20 AM  08:00 PM  Waning Gibbous  Louisville 

=== Air Quality ===
    PM2.5    PM10     O3   NO2   SO2      CO        City
0  14.245  14.615  122.0  9.62  4.44  390.35  Louisville 

